In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [2]:
#Feature Film, Released between 2014-11-01 and 2019-10-31, Rating Count at least 1,000, United States, English,
#Including Adult Titles (Sorted by Release Date Descending)

In [3]:
base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2014-11-01,2019-10-31&num_votes=1000,&countries=us&languages=en&adult=include&view=simple&sort=release_date,desc&count=250"
mid_url = "&start="
end_url = "&ref_=adv_nxt"

In [4]:
url = base_url
response = get(url)
print(response)

<Response [200]>


In [5]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [6]:
movie_list = html_soup.find_all('div', class_ = 'lister-item mode-simple')
len(movie_list)

250

In [7]:
movie_list[0]

<div class="lister-item mode-simple">
<div class="lister-item-image">
<a href="/title/tt5606664/"> <img alt="Doctor Sleep" class="loadlate" data-tconst="tt5606664" height="50" loadlate="https://m.media-amazon.com/images/M/MV5BYmY3NGJlYTItYmQ4OS00ZTEwLWIzODItMjMzNWU2MDE0NjZhXkEyXkFqcGdeQXVyMzQzMDA3MTI@._V1_UX34_CR0,0,34,50_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="34"/>
</a> </div>
<div class="lister-item-content">
<div class="lister-col-wrapper">
<div class="col-title">
<span class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<span title="Mike Flanagan (dir.), Ewan McGregor, Rebecca Ferguson">
<a href="/title/tt5606664/">Doctor Sleep</a>
<span class="lister-item-year text-muted unbold">(2019)</span>
</span>
</span>
</div>
<div class="col-imdb-rating">
<strong title="7.7 base on 18,244 votes">
                            7.7
                        </strong>
</div>
<div

In [8]:
movie_title = str(movie_list[0].find_all("a")[1]).split("/")[2]

In [9]:
movie_name = movie_list[0].find_all("a")[1].text

In [10]:
movie_title

'tt5606664'

In [11]:
movie_name

'Doctor Sleep'

In [12]:
titles = html_soup.find('div', class_ = 'desc')
title_size = int(titles.span.text.split(" ")[2].replace(",", ""))
title_size

2158

In [13]:
url = base_url
titleID = []
title = []
response_dict = {}
i = 1
while i < title_size:
    response = get(url)
    response_dict[i+249] = response
    movies = BeautifulSoup(response.text, 'html.parser')
    movie_list = movies.find_all('div', class_ = 'lister-item mode-simple')
    for x in range(len(movie_list)):
        titleID.append(str(movie_list[x].find_all("a")[1]).split("/")[2])
        title.append(movie_list[x].find_all("a")[1].text)
    i += 249 #next 250 results
    if i > title_size:
        url = base_url+mid_url+str(title_size)+end_url
    else:
        url = base_url+mid_url+str(i)+end_url
        
print(len(titleID))
print(len(title))

2166
2166


In [14]:
response_dict #check if all successful

{250: <Response [200]>,
 499: <Response [200]>,
 748: <Response [200]>,
 997: <Response [200]>,
 1246: <Response [200]>,
 1495: <Response [200]>,
 1744: <Response [200]>,
 1993: <Response [200]>,
 2242: <Response [200]>}

In [15]:
df_titleID = pd.DataFrame(data = {"titleID": titleID, "title": title})
df_titleID.head() #all USA English movies for the past 5 years

,titleID,title
0,tt5606664,Doctor Sleep
1,tt9257484,Rattlesnake
2,tt6924650,Midway
3,tt10039344,Countdown
4,tt6450804,Terminator: Dark Fate


In [16]:
df_titleID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166 entries, 0 to 2165
Data columns (total 2 columns):
titleID    2166 non-null object
title      2166 non-null object
dtypes: object(2)
memory usage: 34.0+ KB


In [17]:
df_imdb_full = pd.read_csv("/Users/valmadrid/Downloads/title.basics.tsv", sep ="\t")

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
df_imdb_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6293397 entries, 0 to 6293396
Data columns (total 9 columns):
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           int64
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtypes: int64(1), object(8)
memory usage: 432.1+ MB


In [19]:
q = """SELECT i.*, t.title
    FROM df_titleID as t
    JOIN df_imdb_full as i
    ON t.titleID = i.tconst;"""
df_imdb= pysqldf(q)
df_imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title
0,tt5606664,movie,Doctor Sleep,Doctor Sleep,0,2019,\N,151,"Drama,Fantasy,Horror",Doctor Sleep
1,tt9257484,movie,Rattlesnake,Rattlesnake,0,2019,\N,85,"Drama,Horror,Mystery",Rattlesnake
2,tt6924650,movie,Midway,Midway,0,2019,\N,138,"Action,Drama,History",Midway
3,tt10039344,movie,Countdown,Countdown,0,2019,\N,90,"Horror,Thriller",Countdown
4,tt6450804,movie,Terminator: Dark Fate,Terminator: Dark Fate,0,2019,\N,128,"Action,Adventure,Sci-Fi",Terminator: Dark Fate
...,...,...,...,...,...,...,...,...,...,...
2161,tt2937898,movie,A Most Violent Year,A Most Violent Year,0,2014,\N,125,"Action,Crime,Drama",A Most Violent Year
2162,tt2300975,movie,Jessabelle,Jessabelle,0,2014,\N,90,"Horror,Thriller",Jessabelle
2163,tt1725986,movie,Some Kind of Beautiful,Some Kind of Beautiful,0,2014,\N,99,"Comedy,Romance",Lessons in Love
2164,tt0900387,movie,Suite Française,Suite Française,0,2014,\N,107,"Drama,Romance,War",Suite Française


In [20]:
for column in ["titleType", "startYear", "genres"]:
    display(df_imdb[column].value_counts())

movie    2166
Name: titleType, dtype: int64

2017    489
2016    488
2015    467
2018    440
2019    233
2014     49
Name: startYear, dtype: int64

Drama                         117
Comedy                        103
Comedy,Drama                   94
Horror,Thriller                81
Comedy,Drama,Romance           66
                             ... 
Comedy,Family,Romance           1
Comedy,Mystery,Sci-Fi           1
Action,Thriller,Western         1
Adventure,Biography,Comedy      1
Fantasy,Horror,Musical          1
Name: genres, Length: 254, dtype: int64

In [21]:
df_imdb["check"] = df_imdb["primaryTitle"] == df_imdb["title"]

In [22]:
df_imdb.startYear = df_imdb.startYear.astype("int64")

In [23]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166 entries, 0 to 2165
Data columns (total 11 columns):
tconst            2166 non-null object
titleType         2166 non-null object
primaryTitle      2166 non-null object
originalTitle     2166 non-null object
isAdult           2166 non-null int64
startYear         2166 non-null int64
endYear           2166 non-null object
runtimeMinutes    2166 non-null object
genres            2166 non-null object
title             2166 non-null object
check             2166 non-null bool
dtypes: bool(1), int64(2), object(8)
memory usage: 171.5+ KB


In [24]:
df_imdb_final= df_imdb.query("(titleType == 'movie') & (startYear >= 2014) & (check == True)").loc[df_imdb["genres"] != "\\N"]
df_imdb_final

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,check
0,tt5606664,movie,Doctor Sleep,Doctor Sleep,0,2019,\N,151,"Drama,Fantasy,Horror",Doctor Sleep,True
1,tt9257484,movie,Rattlesnake,Rattlesnake,0,2019,\N,85,"Drama,Horror,Mystery",Rattlesnake,True
2,tt6924650,movie,Midway,Midway,0,2019,\N,138,"Action,Drama,History",Midway,True
3,tt10039344,movie,Countdown,Countdown,0,2019,\N,90,"Horror,Thriller",Countdown,True
4,tt6450804,movie,Terminator: Dark Fate,Terminator: Dark Fate,0,2019,\N,128,"Action,Adventure,Sci-Fi",Terminator: Dark Fate,True
...,...,...,...,...,...,...,...,...,...,...,...
2160,tt3254796,movie,Big Stone Gap,Big Stone Gap,0,2014,\N,103,"Comedy,Romance",Big Stone Gap,True
2161,tt2937898,movie,A Most Violent Year,A Most Violent Year,0,2014,\N,125,"Action,Crime,Drama",A Most Violent Year,True
2162,tt2300975,movie,Jessabelle,Jessabelle,0,2014,\N,90,"Horror,Thriller",Jessabelle,True
2164,tt0900387,movie,Suite Française,Suite Française,0,2014,\N,107,"Drama,Romance,War",Suite Française,True


In [25]:
for column in ["titleType", "startYear", "genres"]:
    display(df_imdb_final[column].value_counts())

movie    2015
Name: titleType, dtype: int64

2017    454
2016    452
2015    426
2018    415
2019    224
2014     44
Name: startYear, dtype: int64

Drama                        112
Comedy                       101
Comedy,Drama                  86
Horror,Thriller               76
Horror                        64
                            ... 
Animation,Biography,Crime      1
Crime,Romance                  1
Action,Drama,Romance           1
Crime,Mystery,Sci-Fi           1
Drama,Thriller,War             1
Name: genres, Length: 246, dtype: int64

In [26]:
df_imdb_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 0 to 2165
Data columns (total 11 columns):
tconst            2015 non-null object
titleType         2015 non-null object
primaryTitle      2015 non-null object
originalTitle     2015 non-null object
isAdult           2015 non-null int64
startYear         2015 non-null int64
endYear           2015 non-null object
runtimeMinutes    2015 non-null object
genres            2015 non-null object
title             2015 non-null object
check             2015 non-null bool
dtypes: bool(1), int64(2), object(8)
memory usage: 175.1+ KB


In [27]:
df_imdb_final.to_csv('2016_2019_USmovies.csv', index = False)

In [28]:
df_imdb_final = pd.read_csv('2016_2019_USmovies.csv')
df_imdb_final.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,check
0,tt5606664,movie,Doctor Sleep,Doctor Sleep,0,2019,\N,151,"Drama,Fantasy,Horror",Doctor Sleep,True
1,tt9257484,movie,Rattlesnake,Rattlesnake,0,2019,\N,85,"Drama,Horror,Mystery",Rattlesnake,True
2,tt6924650,movie,Midway,Midway,0,2019,\N,138,"Action,Drama,History",Midway,True
3,tt10039344,movie,Countdown,Countdown,0,2019,\N,90,"Horror,Thriller",Countdown,True
4,tt6450804,movie,Terminator: Dark Fate,Terminator: Dark Fate,0,2019,\N,128,"Action,Adventure,Sci-Fi",Terminator: Dark Fate,True


In [29]:
df_ratings = pd.read_csv("/Users/valmadrid/Downloads/title.ratings.tsv", sep ="\t")
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1547
1,tt0000002,6.1,187
2,tt0000003,6.5,1204
3,tt0000004,6.2,114
4,tt0000005,6.1,1932


In [30]:
q = """SELECT imdb.*, r.averageRating, r.numVotes
    FROM df_ratings as r
    JOIN df_imdb_final as imdb
    USING (tconst);"""
df_imdb_ratings = pysqldf(q)
df_imdb_ratings

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,check,averageRating,numVotes
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,The Other Side of the Wind,1,6.9,4989
1,tt0339736,movie,The Evil Within,The Evil Within,0,2017,\N,98,Horror,The Evil Within,1,5.5,2572
2,tt0360556,movie,Fahrenheit 451,Fahrenheit 451,0,2018,\N,100,"Drama,Sci-Fi,Thriller",Fahrenheit 451,1,4.9,15295
3,tt0365545,movie,Nappily Ever After,Nappily Ever After,0,2018,\N,98,"Comedy,Drama,Romance",Nappily Ever After,1,6.4,6718
4,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",Jurassic World,1,7.0,549806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,tt9562694,movie,Alien Warfare,Alien Warfare,0,2019,\N,88,"Action,Sci-Fi",Alien Warfare,1,2.6,2003
2011,tt9664078,movie,American Son,American Son,0,2019,\N,90,Drama,American Son,1,5.6,2233
2012,tt9815714,movie,The Hard Way,The Hard Way,0,2019,\N,92,Action,The Hard Way,1,4.8,1717
2013,tt9831136,movie,The Banana Splits Movie,The Banana Splits Movie,0,2019,\N,89,"Comedy,Horror,Sci-Fi",The Banana Splits Movie,1,5.0,1668


In [37]:
df_imdb_ratings.to_csv('2016_2019_USmovies.csv', index = False)

In [39]:
df_imdb_ratings = pd.read_csv('2016_2019_USmovies.csv')
df_imdb_ratings

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,check,averageRating,numVotes
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,The Other Side of the Wind,1,6.9,4989
1,tt0339736,movie,The Evil Within,The Evil Within,0,2017,\N,98,Horror,The Evil Within,1,5.5,2572
2,tt0360556,movie,Fahrenheit 451,Fahrenheit 451,0,2018,\N,100,"Drama,Sci-Fi,Thriller",Fahrenheit 451,1,4.9,15295
3,tt0365545,movie,Nappily Ever After,Nappily Ever After,0,2018,\N,98,"Comedy,Drama,Romance",Nappily Ever After,1,6.4,6718
4,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",Jurassic World,1,7.0,549806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,tt9562694,movie,Alien Warfare,Alien Warfare,0,2019,\N,88,"Action,Sci-Fi",Alien Warfare,1,2.6,2003
2011,tt9664078,movie,American Son,American Son,0,2019,\N,90,Drama,American Son,1,5.6,2233
2012,tt9815714,movie,The Hard Way,The Hard Way,0,2019,\N,92,Action,The Hard Way,1,4.8,1717
2013,tt9831136,movie,The Banana Splits Movie,The Banana Splits Movie,0,2019,\N,89,"Comedy,Horror,Sci-Fi",The Banana Splits Movie,1,5.0,1668


In [41]:
df_name= pd.read_csv("/Users/valmadrid/Downloads/name.basics.tsv", sep ="\t")
df_name.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0043044,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0071877,tt0117057,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0059956,tt0054452,tt0057345,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0078723,tt0077975,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0069467,tt0050986,tt0050976"


In [31]:
base_url1 = "https://www.imdb.com/title/"
end_url1 = "/?ref_=adv_li_tt"

In [32]:
url = base_url1+df_imdb_final.tconst.iloc[6]+end_url1
response = get(url)
print(response)

<Response [200]>


In [33]:
html_soup = BeautifulSoup(response.text, 'html.parser')
movie_info = html_soup
movie_info


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///title/tt4777008?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Maleficent: Mistress of Evil (2019) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href=

In [34]:
movie = movie_info.find_all("div", class_ = "txt-block")
movie

[<div class="txt-block">
 <h4 class="inline">Taglines:</h4>
 On October 18, go beyond the fairy tale.        </div>,
 <div class="txt-block">
 <h4 class="inline">Certificate:</h4>
 <span>PG</span>
 <span class="ghost">|</span> <span class="see-more inline">
 <a href="/title/tt4777008/parentalguide#certification"> See all certifications</a> »
             </span>
 </div>,
 <div class="txt-block">
 <h4 class="inline">Parents Guide:</h4>
 <span class="see-more inline">
 <a href="/title/tt4777008/parentalguide"> View content advisory</a> »
         </span>
 </div>,
 <div class="txt-block">
 <h4 class="inline">Official Sites:</h4>
 <a href="/offsite/?page-action=offsite-isputlockers&amp;token=BCYvz7hq9n6IrDXYsBQGCyRvhD2s7Svu4WjappQ_017PgeX6-OUK1due8bIZBBJKBrARyYtpeZlW%0D%0AJu806AVgxsm5xdw17-78TRfKu1zW9F_EgRyrRjNG18PM0-q16fcwMCo6h0C6m-ypL4zedRkqm40W%0D%0A2Qb-B45_Ehu068jwwRGW0QLb-nNAXBJ6Nyh6H-q1TN-OJ7gYQk0fqHcu6qtG04zjV8wiVm9B1o0O%0D%0AFjywuWLbAMyLoTDQSKUeTeny7aL3-A_o%0D%0A" rel="nofollow">is

In [35]:
movie_text = []
for i in range(len(movie)):
    new_string = movie[i].text.replace("\n", "").strip()
    movie_text.append(new_string)

In [36]:
movie_text

['Taglines:On October 18, go beyond the fairy tale.',
 'Certificate:PG|  See all certifications\xa0»',
 'Parents Guide: View content advisory\xa0»',
 'Official Sites:isputlockers|Official Site',
 'Country:USA|UK',
 'Language:English',
 'Release Date: 18 October 2019 (UK)    See more\xa0»',
 'Also Known As: Maleficent: Mistress of Evil      See more\xa0»',
 'Filming Locations:Pinewood Studios, Iver Heath, Buckinghamshire, England, UKSee more\xa0»',
 'Budget:$185,000,000            (estimated)',
 'Opening Weekend USA: $36,948,713,20 October 2019',
 'Gross USA: $97,714,087',
 'Cumulative Worldwide Gross: $431,883,192',
 'Production Co: Roth Films, Walt Disney Pictures See more\xa0»',
 'Show more on  IMDbPro\xa0»',
 'Runtime:119 min',
 'Sound Mix:Dolby Atmos| Dolby Digital| DTS(DTS: X)| Dolby Surround 7.1| Auro 11.1| IMAX 6-Track| 12-Track Digital Sound| Sonics-DDP',
 'Color:Color',
 'Aspect Ratio: 2.39 : 1',
 'Trivia    With the addition of this film, there is a grand total of 10 films re